In [ ]:
pip install folium

In [ ]:
import folium

In [ ]:
!pip install psycopg2-binary

In [3]:
import psycopg2

In [4]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")

In [5]:
db = conn.cursor()
#query point : 105.8475554, 21.007130000404484
#test point :105.85062869999999 21.00922750040452

# IER

In [28]:
def IED(long, lat):
    path=[]
    #find euclide NN
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                limit 1""")
    qPoint=db.fetchall()
    #find network routing
    #qPoint[0][0],qPoint[0][1]: longtitude and latitude of NN point 
    db.execute(f"""
                WITH start AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                      LIMIT 1
                    ),
                destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({qPoint[0][0]} {qPoint[0][1]})'),
                      4326)
                  LIMIT 1
                    )
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    DEmax= result[0][0]
    path= result[0][1]
    #find candidate in this network routing
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<=  {DEmax}
                """)
    NN=db.fetchall()
    #find result
    for i in NN:
        db.execute(f"""
                WITH start AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long} {lat})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT ST_AsText(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][1]>DEmax) : continue
        else:
            path=x[0][0]
            DEmax =x[0][1]
            qPoint=i
    return qPoint, DEmax, path 

In [29]:
IED(105.8042238, 21.030628300404896)

((105.8068682, 21.033045300404943),
 471.871298337968,
 'MULTILINESTRING((105.8035522 21.0317839,105.8036057 21.031756,105.8037051 21.0317041,105.8041041 21.0314911,105.8042574 21.0314584,105.8043078 21.0314273,105.8044495 21.031407,105.8045519 21.0314498,105.8046592 21.0315475,105.8047719 21.0316401,105.8048496 21.0316927,105.8048891 21.0316887,105.8050964 21.0316676,105.8051715 21.0316651,105.8053484 21.0316726,105.805693 21.0321733),(105.805693 21.0321733,105.8059144 21.0326189),(105.8059144 21.0326189,105.8060016 21.0327537,105.8061194 21.0327128,105.8061471 21.0327422),(105.8061471 21.0327422,105.8062683 21.032994),(105.8062683 21.032994,105.8064193 21.0333239,105.806441 21.0333701))')

In [ ]:
POINT(105.8367243 20.991411700404193)

In [ ]:
POINT(105.8502594 20.997397700404303)

In [ ]:
#NN p1
IED(105.8367243, 20.991411700404193)

In [ ]:
#NN p2
IED(105.8502594, 20.997397700404303)

# NN to 2 given point

## method 1 candidat by IER

In [24]:
# find NN by IER
def givenP(long, lat):
    path=[]
    #find euclide NN
    db.execute(f"""select long, lat FROM geometries_vn
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
    qPoint=db.fetchall()
    #find network routing
    db.execute(f"""
                WITH start AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                      LIMIT 1
                    ),
                destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({qPoint[0][0]} {qPoint[0][1]})'),
                      4326)
                  LIMIT 1
                    )
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    DEmax= result[0][0]
    path= result[0][1]
    #find candidate in this network routing
    db.execute(f"""select long, lat FROM geometries_vn
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<=  {DEmax}
                """)
    NN=db.fetchall()
    #find result
    for i in NN:
        db.execute(f"""
                WITH start AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long} {lat})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT (ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][1]>DEmax) : continue
        else:
            path=x[0][0]
            DEmax =x[0][1]
            qPoint=i
    return qPoint, DEmax, path 

In [5]:
#givenP(105.8367243, 20.991411700404193)
##NN1=

In [6]:
#givenP(105.8502594, 20.997397700404303)
#NN2=

In [25]:
#calculating the cost by network routing from NN above to 2 given point

def cost2point(long, lat, long1, lat1, long2, lat2):
    db.execute(f"""
                WITH start1 AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long1} {lat1})'),
                      4326)
                      LIMIT 1
                    ),
                start2 AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long2} {lat2})'),
                      4326)
                  LIMIT 1
                    ),
                    destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                  LIMIT 1
                    )
                    
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start1 UNION SELECT source from start2),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    cost= result[0][0]
    return cost

In [26]:
cost2point(105.8457825, 20.997738100404312,105.8367243, 20.991411700404193,105.8502594, 20.997397700404303)
#sum cost of NN1

2394.4960088968674

In [27]:
cost2point(105.8503479, 20.99818290040432,105.8367243, 20.991411700404193,105.8502594, 20.997397700404303)
#sum_cost NN2

2529.2696374778766

In [28]:
#cost by NN1 is smaller, then => query point from point2 with euclide = NN1, 105.84089229999998, 20.995627400404274

In [29]:
from functools import reduce

In [30]:
#finding all candidate which is within smaller network routing distance

def candidate(long1,lat1,long2,lat2,max1):
#     max1=2394.4960088968674
#     max2=2529.2696374778766
    waste=0
    can=[]
    path=1
    result=1
    end=[]
    db.execute(f"""
    select long, lat FROM geometries
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long2}, {lat2}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    qPoint1=db.fetchall()
    db.execute(f"""
    select long, lat FROM geometries
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long1}, {lat1}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    qPoint2=db.fetchall()
# #     db.execute(f"""
# #     select long, lat FROM geometries
# #                 where amenity='fast_food'
# #                 AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
# #                 AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long1}, {lat1}), 4326), 3857), 
# #                                 ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
# #                 """)
# #     qPoint1=db.fetchall()
    intersection = reduce(lambda acc, x: acc + [x] if x in qPoint1 and x not in acc else acc, qPoint2, [])
#     #find network routing
    for i in intersection:
        db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT ST_AsText(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        #end.append(x[0][1])
        if(x[0][1]>max1) : continue
        else:
            path=x[0][0]
            waste =x[0][1]
            result=i
    return result, waste, path

    #return end

#### My function

In [31]:
def path2point(long1, lat1, long2,lat2):
    NN1=givenP(long1,lat1)
    NN2=givenP(long2,lat2)
    x1=cost2point(NN1[0][0],NN1[0][1],long1, lat1, long2,lat2)
    x2=cost2point(NN2[0][0],NN2[0][1],long1, lat1, long2,lat2)
    if x1>=x2:
        result=candidate(long1, lat1, long2,lat2,x2)
    else: result=candidate(long1, lat1, long2,lat2,x1)
    return result

In [32]:
# need to build filter by NN euclide and then overlap

In [33]:
path2point(105.8367243, 20.991411700404193,105.8502594, 20.997397700404303)

((105.84603819999998, 20.996801500404292),
 2120.7846004646144,
 'MULTILINESTRING((105.8499573 20.9961284,105.8499753 20.9962416,105.8500515 20.9968868),(105.8499573 20.9961284,105.8497983 20.9961312,105.849507 20.9961644),(105.849507 20.9961644,105.8488094 20.996249),(105.8488094 20.996249,105.848604 20.9962714),(105.848604 20.9962714,105.8479447 20.9963545,105.8473146 20.9964319),(105.8473146 20.9964319,105.8471702 20.9964571),(105.8471702 20.9964571,105.8467169 20.9965467,105.8466767 20.9965546,105.8466055 20.9965725),(105.8367366 20.9924172,105.8368404 20.9918063),(105.8366609 20.9928628,105.8366681 20.9928202,105.8367366 20.9924172),(105.8366609 20.9928628,105.8366981 20.9928678),(105.8366981 20.9928678,105.8368966 20.9928904),(105.8368966 20.9928904,105.8369411 20.9928943),(105.8370216 20.9929323,105.8369411 20.9928943),(105.8373479 20.9930114,105.837094 20.9929565,105.8370548 20.992948,105.8370216 20.9929323),(105.8372977 20.9933642,105.8373265 20.9931617,105.8373479 20.9930114)

## method 2:candidat by euclide

In [34]:
from functools import reduce

In [35]:
def euclid(long,lat):
    db.execute(f"""select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries_vn
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
    NN=db.fetchall()
    return NN

In [36]:
x1=euclid(105.8367243, 20.991411700404193)
x1

[(105.84415592757635, 20.99778753353857, 1123.538463710309)]

In [37]:
x2=euclid(105.8502594, 20.997397700404303)
x2

[(105.85063121852589, 20.999756697279977, 284.31149388834945)]

In [38]:
def find_NN(long,lat,max1):   
    db.execute(f"""
    select long, lat FROM geometries_vn
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    Point=db.fetchall()
    return Point

In [39]:
NN1=find_NN(105.8367243, 20.991411700404193,x1[0][2])
NN1

[(105.84415592757635, 20.99778753353857)]

In [40]:
NN2=find_NN(105.8502594, 20.997397700404303,x1[0][2])
NN2

[(105.85063121852589, 20.999756697279977),
 (105.84408432517345, 20.997963931988973),
 (105.84415592757635, 20.99778753353857),
 (105.84333377184484, 21.002040406245815)]

In [41]:
intersection = reduce(lambda acc, x: acc + [x] if x in NN1 and x not in acc else acc, NN2, [])
intersection

[(105.84415592757635, 20.99778753353857)]

In [42]:
long1=105.8367243
lat1= 20.991411700404193
long2=105.8502594
lat2=20.997397700404303
db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({intersection[0][0]} {intersection[0][1]})'),4326)
                          LIMIT 1
                                    )
                SELECT (ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
x=db.fetchall()

In [58]:
a=set(NN1).union(NN2)
for i in a:
    print(i, i[0],i[1])

(105.85063121852589, 20.999756697279977) 105.85063121852589 20.999756697279977
(105.84333377184484, 21.002040406245815) 105.84333377184484 21.002040406245815
(105.84408432517345, 20.997963931988973) 105.84408432517345 20.997963931988973
(105.84415592757635, 20.99778753353857) 105.84415592757635 20.99778753353857


In [43]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")
db = conn.cursor()

In [59]:
#find best result in candidate set
def intersection(long1, lat1, long2, lat2):
    x1= euclid(long1,lat1)
    x2= euclid(long2, lat2)
    result=[]
    if x1[0][2] > x2[0][2]:
        max_e=x1[0][2]
    else:
        max_e=x2[0][2]
    NN1= find_NN(long1,lat1, max_e)
    NN2= find_NN(long2, lat2,max_e)
    #intersection = reduce(lambda acc, x: acc + [x] if x in NN1 and x not in acc else acc, NN2, [])
    union = set(NN1).union(NN2)
    result=[]
    for i in union:
        db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT  {i} as point,(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][-1] is not None):       
            result.append(x)
            
    sorted_x= sorted(result, key=lambda x: x[0][-1])
    return sorted_x[0]
    #return result[0]

### my function

In [60]:
result=intersection(long1=105.8367243,
lat1= 20.991411700404193,
long2=105.8502594,
lat2=20.997397700404303)

In [61]:
result

[('(105.84415592757635,20.99778753353857)',
  '0105000020E61000002E000000010200000003000000C69F4DB365765A40E79D544502FF344019FACCFE65765A402BEA82B009FF34409C1A683E67765A4068D02BF933FF3440010200000003000000C69F4DB365765A40E79D544502FF344092AC681863765A4076864E7402FF344047AE9B525E765A4081704FA104FF344001020000000200000047AE9B525E765A4081704FA104FF34406462A9E452765A4036ACA92C0AFF34400102000000020000006462A9E452765A4036ACA92C0AFF34404ED026874F765A40ADF078A40BFF34400102000000030000004ED026874F765A40ADF078A40BFF344042D4D8B944765A40F1B8A81611FF3440181B04673A765A40EF30372916FF3440010200000002000000181B04673A765A40EF30372916FF344054C95B0938765A40F45D00D017FF344001020000000400000054C95B0938765A40F45D00D017FF34401D9E149C30765A40CD6F3DAF1DFF3440413278F32F765A40A9ADC7331EFF344047C7D5C82E765A40CC9717601FFF344001020000000500000047C7D5C82E765A40CC9717601FFF34400E7915632A765A40C923134B25FF3440097DFA1928765A4039D384ED27FF3440ED832C0B26765A40977329AE2AFF3440415AAD5D24765A40FB18BD642DFF3440010200000004000

### other: not intersect, just union

In [62]:
unique_elements = set(NN1).intersection(NN2)
unique_list = list(unique_elements)

In [63]:
unique_list

[(105.84415592757635, 20.99778753353857)]

In [67]:
test_result=[]
for i in unique_list:
    db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT {i},(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
    x=db.fetchall()
    test_result.append(x)

In [68]:
test_result
test_result[0][0][-1]

2403.5159154805397

In [69]:
sorted_list = sorted(test_result, key=lambda x: x[0][-1])
sorted_list

[[('(105.84415592757635,20.99778753353857)',
   '0105000020E61000002E000000010200000003000000C69F4DB365765A40E79D544502FF344019FACCFE65765A402BEA82B009FF34409C1A683E67765A4068D02BF933FF3440010200000003000000C69F4DB365765A40E79D544502FF344092AC681863765A4076864E7402FF344047AE9B525E765A4081704FA104FF344001020000000200000047AE9B525E765A4081704FA104FF34406462A9E452765A4036ACA92C0AFF34400102000000020000006462A9E452765A4036ACA92C0AFF34404ED026874F765A40ADF078A40BFF34400102000000030000004ED026874F765A40ADF078A40BFF344042D4D8B944765A40F1B8A81611FF3440181B04673A765A40EF30372916FF3440010200000002000000181B04673A765A40EF30372916FF344054C95B0938765A40F45D00D017FF344001020000000400000054C95B0938765A40F45D00D017FF34401D9E149C30765A40CD6F3DAF1DFF3440413278F32F765A40A9ADC7331EFF344047C7D5C82E765A40CC9717601FFF344001020000000500000047C7D5C82E765A40CC9717601FFF34400E7915632A765A40C923134B25FF3440097DFA1928765A4039D384ED27FF3440ED832C0B26765A40977329AE2AFF3440415AAD5D24765A40FB18BD642DFF34400102000000040

# with a set of point

In [70]:
import psycopg2
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")
db = conn.cursor()

In [71]:
db.execute(f"""
    select long, lat from geometries_vn
    where amenity ='hospital'
    AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE name = 'Quận Hoàn Kiếm'))
""")
h=db.fetchall()

In [72]:
import itertools
for b in h:
    print(b)

(105.85294349978018, 21.02541801364248)
(105.85014384759315, 21.036902233378417)


### my function

In [73]:
dis=0
end=[]
candidat=[]

def bePoint():
    db.execute(f"""
    select long, lat from geometries
    where amenity ='hospital'
    AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE osm_id='-9421131'))
""")
    h=db.fetchall()
    for b in h:
        b=b
        db.execute(f"""
    select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(105.8548261, 21.024432000404786), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(105.8548261, 21.024432000404786), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                limit 1
    
    """)
    
        r= db.fetchall()
#     res.append(x)
#     sorted_res = sorted(res, key=lambda x: x[0][-1])
    
        db.execute(f"""select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries
                where amenity='atm'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
        a= db.fetchall() 
    
        if r[0][2]> a[0][2]:
            dis= r[0][2]
        else:
            dis= a[0][2]
        
        db.execute(f"""
    select long, lat
    FROM geometries
                where amenity='hospital'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<={dis}  
    """)
        h_nn= db.fetchall()
    
        db.execute(f"""
    select long, lat
    FROM geometries
                where amenity='atm'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<={dis}  
    """)
        a_nn= db.fetchall()
    
        for i, z in itertools.product(h_nn,a_nn):
            db.execute(f"""
            WITH start1 AS (
          SELECT topo.source
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
        ST_GeomFromText('POINT({r[0][0]} {r[0][1]})'),4326) 
          LIMIT 1)

        ,start2 as (
        SELECT topo.source
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
            ST_GeomFromText('POINT({z[0]} {z[1]})'),4326) 
          LIMIT 1)
        ,destination AS (
          SELECT topo.source 
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
        ST_GeomFromText('POINT({i[0]} {i[1]})'),4326) 
          LIMIT 1
        )
        SELECT ST_Union(geom_way), sum(di.cost) as realcost, {i}, {z},{r[0]}
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 ),
                                    array(SELECT source FROM destination UNION SELECT source FROM start2),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
            c= db.fetchall()     
#         Some point has not path and cost, it means they are not reach to others
#         if c[0][0] is not None:  
            candidat.append(c)
        end= sorted(candidat, key=lambda x: x[0][1])[0]
# #     candidat1= sorted_c
#     end.append(candidat)
    return end

In [131]:
bePoint() #path, cost, hospital, atm, restaurant

[('0102000020E61000000400000004BBAB68BD765A40126E8D637F0635406A6CAF05BD765A40BBAAFDE77A063540CA07F30DBB765A40A314BE0864063540BE45CC91BA765A400BF20E4B5E063540',
  63.24046506662777,
  '(105.8548261,21.024432000404786)',
  '(105.8553356,21.024990100404796)',
  '(105.8549087,21.024559900404793,17.810343901958767)')]

In [74]:
bePoint()

[('0102000020E61000000400000004BBAB68BD765A40126E8D637F0635406A6CAF05BD765A40BBAAFDE77A063540CA07F30DBB765A40A314BE0864063540BE45CC91BA765A400BF20E4B5E063540',
  63.24046506662777,
  '(105.8548261,21.024432000404786)',
  '(105.8553356,21.024990100404796)',
  '(105.8549087,21.024559900404793,17.810343901958767)')]

In [ ]:
# #ST_AsText(ST_Union(geom_way)),sum(di.cost) as realcost
# def realcost(long2,lat2):
#     db.execute(f"""
#     WITH start AS (
#   SELECT topo.source 
#   FROM osm_2po_4pgr as topo
#   ORDER BY topo.geom_way <-> ST_SetSRID(
#     ST_GeomFromText('POINT (105.8475554 21.007130000404484)'),
#   4326)
#   LIMIT 1
#     ),
#     destination AS (
#   SELECT topo.source 
#   FROM osm_2po_4pgr as topo
#   ORDER BY topo.geom_way <-> ST_SetSRID(
#     ST_GeomFromText('POINT ({long2} {lat2})'),
#   4326)
#   LIMIT 1
#     )
#     SELECT sum(di.cost) as realcost
#     FROM pgr_dijkstra('
#     SELECT id,
#          source,
#          target,
#          ST_Length(ST_Transform(geom_way, 3857)) AS cost
#         FROM osm_2po_4pgr',
#     array(SELECT source FROM start),
#     array(SELECT source FROM destination),
#     directed := false) AS di
#     JOIN   osm_2po_4pgr AS pt
#   ON   di.edge = pt.id
#   """)
#     cost = db.fetchall()
#     return cost

In [ ]:
db.execute("""SELECT long, lat, ST_Distance( ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(105.8475554, 21.007130000404484), 4326), 3857)) as cost 
            FROM geometries WHERE amenity = 'restaurant' 
           ORDER BY cost 
           LIMIT 1;"""
          )

In [ ]:
NN= db.fetchall()

In [ ]:
def euclidcost(long2, lat2):
    query = f"""
        SELECT ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(105.8475554, 21.007130000404484), 4326), 3857), 
                           ST_Transform(ST_SetSRID(ST_MakePoint({long2}, {lat2}), 4326), 3857)) as cost
        FROM geometries
    """
    db.execute(query)
    cost = db.fetchone()
    return cost

query = f"""
    SELECT long, lat
    FROM geometries
    WHERE amenity = 'restaurant'
    ORDER BY (
        SELECT cost
        FROM (
            SELECT euclidcost(geometries.long, geometries.lat) as cost
            FROM geometries
        ) AS subquery
    )
    LIMIT 1
"""

db.execute(query)


In [ ]:
query = """
    SELECT long, lat
    FROM geometries
    WHERE amenity = 'restaurant'
    ORDER BY euclidcost(105.8475554::numeric, 21.007130000404484::numeric, geometries.long, geometries.lat)
    LIMIT 1
"""

db.execute(query)


In [ ]:
[(105.85062869999999, 21.00922750040452), (105.8496041, 21.010285700404545)]

In [ ]:
db.execute("""
WITH start AS (
  SELECT topo.source --could also be topo.target
  FROM osm_2po_4pgr as topo
  ORDER BY topo.geom_way <-> ST_SetSRID(
    ST_GeomFromText('POINT (105.8475554 21.007130000404484)'),
  4326)
  LIMIT 1
),
destination AS (
  SELECT topo.source --could also be topo.target
  FROM osm_2po_4pgr as topo
  ORDER BY topo.geom_way <-> ST_SetSRID(
    ST_GeomFromText('POINT (105.8496041 21.010285700404545)'),
  4326)
  LIMIT 1
)
SELECT ST_AsText(ST_Union(geom_way)),sum(di.cost) as realcost
FROM pgr_dijkstra('
    SELECT id,
         source,
         target,
         ST_Length(ST_Transform(geom_way, 3857)) AS cost
        FROM osm_2po_4pgr',
    array(SELECT source FROM start),
    array(SELECT source FROM destination),
    directed := false) AS di
JOIN   osm_2po_4pgr AS pt
  ON   di.edge = pt.id
  """)

In [ ]:
DEmax= db.fetchall()

In [ ]:
DEmax

In [ ]:
db.close()

In [ ]:
db.execute("""SELECT long, lat, ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(105.8475554, 21.007130000404484), 4326), 3857)) as cost 
            FROM geometries WHERE amenity = 'restaurant' and 
                                ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(105.8475554, 21.007130000404484), 4326), 3857)) <= 442.6842043808904
                                ORDER BY cost
                                """
                                )

In [ ]:
ANN= db.fetchall()

In [ ]:
ANN

In [ ]:
ST_AsText(ST_Union(geom_way))

In [ ]:
realcost(105.85062869999999, 21.00922750040452)

In [ ]:
euclidcost()

In [ ]:
# print(cursor.fetchone())

In [ ]:
# print(cursor.fetchall())

In [ ]:
# print(cursor.fetchmany(size=5))

In [ ]:
# one = cursor.fetchone()
# all = cursor.fetchall()

In [ ]:
#print(db.fetchmany(size=50))

In [ ]:
# find destination source
cur = conn.cursor()

# Execute the tenNN query
cur.execute("""
    WITH tenNN AS (
        SELECT long, lat
        FROM geometries
        WHERE amenity = 'restaurant'
        ORDER BY ST_Distance(
            ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857),
            ST_Transform(ST_SetSRID(ST_MakePoint(105.8452161, 21.00105440040437), 4326), 3857)
        )
        LIMIT 10
    )
    SELECT long, lat
    FROM tenNN
""")

# Fetch all rows from the query result
rows = cur.fetchall()
destination=[];
# Iterate over the rows and retrieve the corresponding source values
for row in rows:
    long, lat = row
    # Execute the query to retrieve the source value
    cur.execute("""
        SELECT topo.source
        FROM osm_2po_4pgr AS topo
        ORDER BY topo.geom_way <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326)
        LIMIT 1
    """, (long, lat))   
    source=cur.fetchone()[0]
    # Fetch the source value
    destination.append(source)

    # Print the result
    print(f"long: {long}, lat: {lat}, source: {source}")

In [ ]:
cur.close() 
conn.close()

In [ ]:
destination

In [ ]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")

In [ ]:
cur = conn.cursor()
cur.execute("""SELECT source FROM osm_2po_4pgr
                ORDER BY geom_way <-> ST_SetSRID(ST_MakePoint(105.83589009999999, 21.016768700404654), 4326)
                LIMIT 1""")
start=cur.fetchall()[0]

In [ ]:
beginning=[]
for x in start:
    beginning.append(x)

In [ ]:
beginning

In [ ]:
cur.close() 
conn.close()

In [ ]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")
cur = conn.cursor()

In [ ]:
cur.execute("""
            SELECT ST_Union(geom_way), cost AS route
            FROM pgr_dijkstra(
              'SELECT id, source, target, ST_Length(ST_Transform(geom_way, 3857)) AS cost FROM osm_2po_4pgr',
              1817247,
              ARRAY(select source from destination), 
              directed := false
            ) AS di
            JOIN osm_2po_4pgr AS pt 
                ON di.edge = pt.id
            ORDER BY cost
            LIMIT 5
        """)
        
        # Process the query result
result = cur.fetchall()
        # Do something with the result

In [ ]:
import psycopg2

# Establish a connection to the database
conn = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")
cur = conn.cursor()

# Retrieve the ATM and hospital points
cur.execute("SELECT long, lat FROM geometries WHERE amenity = 'atm'")
atm_points = cur.fetchall()

cur.execute("SELECT long, lat FROM geometries WHERE amenity = 'hospital'")
hospital_points = cur.fetchall()

# Loop through each pair of ATM and hospital points
for atm_point in atm_points:
    for hospital_point in hospital_points:
        atm_long, atm_lat = atm_point
        hospital_long, hospital_lat = hospital_point

        # Find the nearest restaurant point for the current pair
        cur.execute("""
            WITH restaurant_distances AS (
                SELECT
                    long,
                    lat,
                    way,
                    ST_Distance(ST_SetSRID(ST_MakePoint(%s, %s), 4326), ST_SetSRID(ST_MakePoint(long, lat), 4326)) AS distance
                FROM
                    geometries
                WHERE
                    amenity = 'restaurant'
            )
            SELECT
                long AS restaurant_long,
                lat AS restaurant_lat,
                way AS restaurant_way
            FROM
                restaurant_distances
            ORDER BY
                distance
            LIMIT 1
        """, (atm_long, atm_lat))

        nearest_restaurant = cur.fetchone()

        # Print the results
        print("ATM Point:", atm_point)
        print("Hospital Point:", hospital_point)
        print("Nearest Restaurant:", nearest_restaurant)
        print("----------------------------------")

# Close the database connection
cur.close()
conn.close()
